Dataset: https://www.kaggle.com/serengil/recognizing-faces-in-the-wild

Source: https://github.com/serengil/tensorflow-101/blob/master/python/H2OAutoMLOverview.ipynb

In [ ]:
!gdown -O dataset.zip https://drive.google.com/uc?id=1U1tjZfl_oiIWzx0FE81SCHYBSufdycYH
!unzip dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1U1tjZfl_oiIWzx0FE81SCHYBSufdycYH
To: /content/dataset.zip
61.9MB [00:00, 64.0MB/s]
Archive:  dataset.zip
  inflating: testset_features.csv    
  inflating: train_true_negative_features.csv  
  inflating: train_true_positive_features.csv  


In [ ]:
!pip install h2o

     |████████████████████████████████| 123.6MB 167kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.6-py2.py3-none-any.whl size=123690782 sha256=d339487e4900d773ca81fd6adf5cfbe36c25149aa0646c65c25657b08910b663
  Stored in directory: /root/.cache/pip/wheels/eb/1a/72/252857fc093eaad70a7f91ddbcabf1dafd38fb5e6c02bcf9b3
Successfully built h2o


In [ ]:
import pandas as pd

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
tp_df = pd.read_csv("train_true_positive_features.csv")
tn_df = pd.read_csv("train_true_negative_features.csv")
df = pd.concat([tp_df, tn_df])
df = df.reset_index(drop = True)

In [ ]:
df = df[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [ ]:
df.sample(5)

,vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
124399,0.557376,1.055818,1.003908,1.416974,0.886460,1.331510,1
129264,0.572976,1.070492,0.916656,1.353998,0.548187,1.047079,1
350452,0.671728,1.159075,1.090128,1.476569,0.825728,1.285090,0
433875,0.546957,1.045903,0.723302,1.202749,0.344244,0.829751,0
13584,0.247176,0.703101,0.439640,0.937699,0.107695,0.464101,1


# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)

# H2O AutoML

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.__version__

'3.26.0.6'

In [ ]:
# specify maximum memory size and number of threads based on your system

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:
import psutil
psutil.virtual_memory()

svmem(total=13655236608, available=12615409664, percent=7.6, used=1197629440, free=8346910720, active=981245952, inactive=3975286784, buffers=75862016, cached=4034834432, shared=913408, slab=221913088)

In [ ]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0y5cawnr
  JVM stdout: /tmp/tmp0y5cawnr/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0y5cawnr/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_unknownUser_pvg9jz
H2O cluster total nodes:,1
H2O cluster free memory:,200 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [ ]:
#pandas to h2o frame
hf = h2o.H2OFrame(x_train)

#alternatively, we can directly import h2o frame instead of converting from pandas
#hf = h2o.import_file('dataset/x_train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.461026,0.960236,0.759346,1.23235,0.380667,0.872545,1
0.523298,1.02303,0.820089,1.28069,0.572227,1.06979,1
0.57931,1.07639,0.834568,1.29195,0.411612,0.907317,1
0.661336,1.15007,1.25314,1.58312,0.809752,1.2726,0
0.380073,0.871864,0.419412,0.915873,0.386296,0.878972,1
0.586928,1.08345,0.980849,1.40061,0.791823,1.25843,0
0.543978,1.04305,0.900194,1.34179,0.88473,1.33021,1
0.378311,0.86984,0.799367,1.26441,0.467552,0.967008,0
0.483638,0.983502,0.590177,1.08644,0.411636,0.907343,1
0.874395,1.32242,0.935428,1.36779,1.11803,1.49535,0


In [ ]:
y_label = "is_related"
x_labels = list(df.drop(columns=[y_label]).columns)

In [ ]:
x_labels

['vgg_cosine',
 'vgg_euclidean_l2',
 'facenet_cosine',
 'facenet_euclidean_l2',
 'openface_cosine',
 'openface_euclidean_l2']

In [ ]:
#this is a binary classification problem. convert is_related column to enum type instead of numerical
#otherwise, it would be a regression problem
hf[y_label] = hf[y_label].asfactor()

In [ ]:
#hf.describe()

In [ ]:
aml = H2OAutoML(max_runtime_secs=60*2)

In [ ]:
%%time
aml.train(x = x_labels, y = y_label, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 1.04 s, sys: 97.4 ms, total: 1.14 s
Wall time: 2min 1s


# Leaderboard

In [ ]:
lb = aml.leaderboard

In [ ]:
#lb.head()
lb.head(rows=lb.nrows)

This H2OFrame is empty.


In [ ]:
saved_model = h2o.save_model(aml.leader, path = "", force=True)

H2OTypeError: ignored

In [ ]:
saved_model

In [ ]:
#you can directly load the best model by running the load_model command
#aml = h2o.load_model("/outputs/sefik/kinship/StackedEnsemble_BestOfFamily_AutoML_20190904_051328")

# Validation Score

In [ ]:
hf_val = h2o.H2OFrame(x_test)

In [ ]:
perf = aml.model_performance(hf_val)

In [ ]:
perf.auc()

In [ ]:
perf.accuracy() [0][1]